In [1]:
from PIL import Image
from pillow_heif import register_heif_opener
import torch
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms 


register_heif_opener()

image = Image.open('data/IMG_1432.HEIC')

print(image.size)
print(image.mode)

(8141, 1917)
RGB


In [2]:
DEVICE = 'mps'

transform = transforms.Compose([ 
    transforms.PILToTensor() 
])

img_tensor = transform(image)

print(img_tensor.shape)
img_tensor.to(DEVICE)

torch.Size([3, 1917, 8141])


tensor([[[224, 223, 223,  ..., 211, 212, 210],
         [225, 223, 223,  ..., 211, 212, 210],
         [224, 223, 224,  ..., 211, 212, 210],
         ...,
         [ 86,  92, 106,  ..., 153, 156, 155],
         [105, 109, 103,  ..., 158, 160, 164],
         [114, 121, 110,  ..., 168, 139, 123]],

        [[233, 232, 232,  ..., 225, 226, 223],
         [234, 232, 232,  ..., 225, 226, 223],
         [233, 232, 233,  ..., 225, 226, 223],
         ...,
         [ 98, 104, 118,  ..., 159, 162, 160],
         [117, 121, 115,  ..., 164, 166, 169],
         [126, 133, 122,  ..., 173, 144, 129]],

        [[240, 239, 239,  ..., 234, 235, 231],
         [241, 239, 239,  ..., 234, 235, 231],
         [240, 239, 240,  ..., 234, 235, 231],
         ...,
         [ 22,  28,  42,  ...,  99, 102, 104],
         [ 41,  45,  39,  ..., 104, 106, 113],
         [ 54,  61,  50,  ..., 117,  88,  81]]], device='mps:0',
       dtype=torch.uint8)

In [3]:
SLICES = 100

# calculate slice width
slice_width = img_tensor.size(2) // SLICES

# slice the image width wise into 100 pieces. discard any remainder  
slices = [img_tensor[:, :, i*slice_width:(i+1)*slice_width] for i in range(SLICES)]

# flatten
flat_slices = [img_slice.reshape(-1) for img_slice in slices]

In [4]:
print(flat_slices[0].shape)

torch.Size([465831])


In [ ]:
flat_slice_size = flat_slices[0].shape
hidden_size = flat_slice_size // 2
latent_size = hidden_size // 2
        
# encoder portion. will take our slices and (learn to) encode them into the latent space
class Encoder(nn.Module):
    def __init__(self):
        self.input_layer = nn.Linear(flat_slice_size, hidden_size)
        self.hidden_layer = nn.Linear(hidden_size, hidden_size)
        self.mean = nn.Linear(hidden_size, latent_size)
        self.var = nn.Linear(hidden_size, latent_size)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.relu(self.input_layer(x))
        x = self.relu(self.hidden_layer(x))
        mean = self.mean(x)
        log_variance = self.var(x)
        
        return mean, log_variance

# decoder portion. takes data that is normally distributed in the latent space
# and decodes to the input space
class Decoder(nn.Module):
    def _init_(self):
        self.latent_in = nn.Linear(latent_size, hidden_size)
        self.hidden_layer = nn.Linear(hidden_size, hidden_size)
        self.output_layer = nn.Linear(hidden_size, flat_slice_size)
        
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.relu(self.latent_in(x))
        x = self.relu(self.hidden_layer(x))
        mean = self.mean(x)
        log_variance = self.var(x)
        
        return mean, log_variance
        